# Imports

In [623]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
import warnings

import joblib

import psycopg2 as ps

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [624]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'

FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')
FILE_JOINED_DATA = os.path.join(PATH_DATA, 'data_joined.csv')
FILE_JOINED_DATA2022 = os.path.join(PATH_DATA, 'data_joined_2022.csv')
FILE_JOINED_DATA2023 = os.path.join(PATH_DATA, 'data_joined_2023.csv')
FILE_JOINED_DATA2024 = os.path.join(PATH_DATA, 'data_joined_2024.csv')

FILE_POSTGRES_PW = os.path.join(PATH_IGNORE, 'postgres-pw.txt')

In [625]:
raw_data = {}

raw_data[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
raw_data[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
raw_data[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Tratando dados
## Padronizando colunas
As colunas são inconsistente entre os datasets. Vamos extrair os dados que queremos um por um, ano por ano.

In [626]:
dfs_yearly = {}

dfs_yearly[2022] = pd.DataFrame()
dfs_yearly[2023] = pd.DataFrame()
dfs_yearly[2024] = pd.DataFrame()

In [627]:
def display_column_from_all(
    col : str,
    head : int = 3
):
    for df in dfs_yearly.values():
        display(df[col].head(head))

In [628]:
def display_value_counts_from_all(
	col : str
):
	for df in dfs_yearly.values():
		print(df[col].value_counts())

In [629]:
def replace_value_with(
	df,
	col,
	value,
	replace_with = np.nan
):
    to_na = df[col] == value # mask
    df.loc[to_na, col] = replace_with    

In [630]:
def extract_data(
    col_name : str,
    col_2022 : str,
    col_2023 : str,
    col_2024 : str
):
	# 2022
	year = 2022
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2022]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2023
	year = 2023
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2023]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2024
	year = 2024
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2024]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

In [631]:
def process_text(
	df,
	col
):
	try:
		df[col] = df[col].str.capitalize()
	except Exception as e:
		print(f'Failed to process text on df[{col}] | {e}')

In [632]:
def process_text_on_all_years(
        col
):
    for year in dfs_yearly:
        df = dfs_yearly[year]
        process_text(df, col)

### Alunos

In [633]:
col = 'aluno'

extract_data(
	col,
	'Nome',
	'Nome Anonimizado',
	'Nome Anonimizado'
)

In [634]:
display_column_from_all(col)

0    Aluno-1
1    Aluno-2
2    Aluno-3
Name: aluno, dtype: object

0    Aluno-861
1    Aluno-862
2    Aluno-863
Name: aluno, dtype: object

0    Aluno-1275
1    Aluno-1276
2    Aluno-1277
Name: aluno, dtype: object

### Ano de nascimento

In [635]:
col = 'ano_nascimento'

extract_data(
	col,
	col_2022 = 'Ano nasc',
	col_2023 = 'Data de Nasc',
	col_2024 = 'Data de Nasc'
)

In [636]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int64

0    6/17/2015
1    5/31/2014
2    2/25/2016
Name: ano_nascimento, dtype: object

0   2016-07-28
1   2016-10-16
2   2016-08-16
Name: ano_nascimento, dtype: datetime64[ns]

In [637]:
dfs_yearly[2022][col] = dfs_yearly[2022][col].astype('int32')

In [638]:
dfs_yearly[2023][col] = pd.to_datetime(dfs_yearly[2023][col])
dfs_yearly[2023][col] = dfs_yearly[2023][col].dt.year

In [639]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].dt.year

In [640]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int32

0    2015
1    2014
2    2016
Name: ano_nascimento, dtype: int32

0    2016
1    2016
2    2016
Name: ano_nascimento, dtype: int32

### INDE

In [641]:
col = 'inde'

extract_data(
    col,
    col_2022 = 'INDE 22',
    col_2023 = 'INDE 2023',
    col_2024 = 'INDE 2024'
)

display_column_from_all('inde')

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2      7.9522
Name: inde, dtype: object

In [642]:
dfs_yearly[2024][col].describe()

count        1092
unique       1055
top       INCLUIR
freq           38
Name: inde, dtype: object

In [643]:
dfs_yearly[2024][col].value_counts()

inde
INCLUIR     38
8.002867     1
7.9522       1
7.156367     1
5.4442       1
            ..
4.78185      1
7.785233     1
8.27555      1
8.0853       1
7.611367     1
Name: count, Length: 1055, dtype: int64

In [644]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR',
    np.nan
)

In [645]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].astype(float)

In [646]:
display_column_from_all(col)

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2    7.952200
Name: inde, dtype: float64

### Pedra 22

In [647]:
col = 'pedra_22'

extract_data(
	col,
	col_2022 = 'Pedra 22',
	col_2023 = 'Pedra 22',
	col_2024 = 'Pedra 22'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0     Quartzo
1    Ametista
2       Ágata
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

pedra_22
Ametista    348
Ágata       250
Quartzo     132
Topázio     130
Name: count, dtype: int64
pedra_22
Ametista    264
Ágata       165
Topázio     118
Quartzo      53
Name: count, dtype: int64
pedra_22
Ametista    218
Ágata       117
Topázio     106
Quartzo      31
Name: count, dtype: int64


### Pedra 23

In [648]:
col = 'pedra_23'

extract_data(
	col,
	col_2022 = 'Pedra 23',
	col_2023 = 'Pedra 2023',
	col_2024 = 'Pedra 23'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 23'
Failed to process text on df[pedra_23] | Can only use .str accessor with string values!


0   NaN
1   NaN
2   NaN
Name: pedra_23, dtype: float64

0    Topázio
1    Topázio
2    Quartzo
Name: pedra_23, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_23, dtype: object

Series([], Name: count, dtype: int64)
pedra_23
Ametista    381
Agata       246
Topázio     232
Quartzo      72
Name: count, dtype: int64
pedra_23
Ametista    308
Topázio     187
Agata       153
Quartzo      42
Name: count, dtype: int64


### Pedra 24

In [649]:
col = 'pedra_23'

extract_data(
	col,
	col_2022 = 'Pedra 24',
	col_2023 = 'Pedra 24',
	col_2024 = 'Pedra 2024'
)

display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 24'
Failed to find column on 2023 | 'Pedra 24'


0   NaN
1   NaN
2   NaN
Name: pedra_23, dtype: float64

0   NaN
1   NaN
2   NaN
Name: pedra_23, dtype: float64

0    Ametista
1     Topázio
2    Ametista
Name: pedra_23, dtype: object

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_23
Ametista    391
Topázio     326
Agata       225
Quartzo     112
INCLUIR      38
Name: count, dtype: int64


In [650]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR'
)

In [651]:
display_value_counts_from_all(
    col
)

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_23
Ametista    391
Topázio     326
Agata       225
Quartzo     112
Name: count, dtype: int64


### Gênero

In [652]:
col = 'genero'

extract_data(
	col,
	col_2022 = 'Gênero',
	col_2023 = 'Gênero',
	col_2024 = 'Gênero'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    Menina
1    Menina
2    Menina
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

genero
Menina    457
Menino    403
Name: count, dtype: int64
genero
Feminino     546
Masculino    468
Name: count, dtype: int64
genero
Feminino     623
Masculino    533
Name: count, dtype: int64


In [653]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
    {
        'Menina':'Feminino',
        'Menino':'Masculino'
	}
)

In [654]:
process_text_on_all_years(col)
display_column_from_all(col)

0    Feminino
1    Feminino
2    Feminino
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

### Ano de ingresso

In [655]:
col = 'ano_ingresso'

extract_data(
	col,
	col_2022 = 'Ano ingresso',
	col_2023 = 'Ano ingresso',
	col_2024 = 'Ano ingresso'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    2016
1    2017
2    2016
Name: ano_ingresso, dtype: int64

0    2023
1    2023
2    2023
Name: ano_ingresso, dtype: int64

0    2024
1    2024
2    2024
Name: ano_ingresso, dtype: int64

ano_ingresso
2022    399
2019    139
2021    138
2018     67
2020     48
2017     40
2016     29
Name: count, dtype: int64
ano_ingresso
2023    369
2022    268
2021    137
2019     98
2018     46
2020     40
2017     32
2016     24
Name: count, dtype: int64
ano_ingresso
2024    349
2021    345
2023    248
2022    214
Name: count, dtype: int64


### Instituição de ensino

In [656]:
col = 'instituicao_ensino'

extract_data(
	col,
	col_2022 = 'Instituição de ensino',
	col_2023 = 'Instituição de ensino',
	col_2024 = 'Instituição de ensino'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0    Escola pública
1      Rede decisão
2      Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Escola pública    752
Rede decisão      106
Escola jp ii        2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


In [657]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
	{
        'Escola pública' : 'Pública',
	}
)

In [658]:
process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0         Pública
1    Rede decisão
2    Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Pública         752
Rede decisão    106
Escola jp ii      2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


### IAN

In [659]:
col = 'ian'

extract_data(
	col,
	col_2022 = 'IAN',
	col_2023 = 'IAN',
	col_2024 = 'IAN'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0     5.0
1    10.0
2    10.0
Name: ian, dtype: float64

0    10.0
1     5.0
2    10.0
Name: ian, dtype: float64

0    10.0
1    10.0
2    10.0
Name: ian, dtype: float64

### IDA

In [660]:
col = 'ida'

extract_data(
	col,
	col_2022 = 'IDA',
	col_2023 = 'IDA',
	col_2024 = 'IDA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.0
1    6.8
2    5.6
Name: ida, dtype: float64

0    9.6
1    8.9
2    6.3
Name: ida, dtype: float64

0    8.0
1    8.0
2    8.0
Name: ida, dtype: float64

### IEG

In [661]:
col = 'ieg'

extract_data(
	col,
	col_2022 = 'IEG',
	col_2023 = 'IEG',
	col_2024 = 'IEG'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.1
1    5.2
2    7.9
Name: ieg, dtype: float64

0    10.0
1     9.1
2     7.6
Name: ieg, dtype: float64

0    8.666667
1    9.333333
2    9.083333
Name: ieg, dtype: float64

### IAA

In [662]:
col = 'iaa'

extract_data(
	col,
	col_2022 = 'IAA',
	col_2023 = 'IAA',
	col_2024 = 'IAA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    8.3
1    8.8
2    0.0
Name: iaa, dtype: float64

0    9.5
1    8.5
2    0.0
Name: iaa, dtype: float64

0    10.002
1    10.002
2    10.002
Name: iaa, dtype: float64

### IPS

In [663]:
col = 'ips'

extract_data(
	col,
	col_2022 = 'IPS',
	col_2023 = 'IPS',
	col_2024 = 'IPS'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    5.6
1    6.3
2    5.6
Name: ips, dtype: float64

0    8.13
1    8.14
2    3.14
Name: ips, dtype: float64

0    6.26
1    3.76
2    3.76
Name: ips, dtype: float64

### IPP

In [664]:
col = 'ipp'

extract_data(
	col,
	col_2022 = 'IPP',
	col_2023 = 'IPP',
	col_2024 = 'IPP'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

Failed to find column on 2022 | 'IPP'


0   NaN
1   NaN
2   NaN
Name: ipp, dtype: float64

0    8.4375
1    7.5000
2    5.9375
Name: ipp, dtype: float64

0    5.625
1    7.500
2    7.500
Name: ipp, dtype: float64

### IPV

In [665]:
col = 'ipv'

extract_data(
	col,
	col_2022 = 'IPV',
	col_2023 = 'IPV',
	col_2024 = 'IPV'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    7.278
1    6.778
2    7.556
Name: ipv, dtype: float64

0    8.920
1    8.585
2    6.260
Name: ipv, dtype: float64

0    5.446667
1    7.050000
2    7.046667
Name: ipv, dtype: float64

### Matemática

In [666]:
col = 'nota_mat'

extract_data(
	col,
	col_2022 = 'Matem',
	col_2023 = 'Mat',
	col_2024 = 'Mat'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    2.7
1    6.3
2    5.8
Name: nota_mat, dtype: float64

0    9.8
1    8.5
2    7.0
Name: nota_mat, dtype: float64

0    10.0
1    10.0
2    10.0
Name: nota_mat, dtype: float64

### Português

In [667]:
col = 'nota_port'

extract_data(
	col,
	col_2022 = 'Portug',
	col_2023 = 'Por',
	col_2024 = 'Por'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    3.5
1    4.5
2    4.0
Name: nota_port, dtype: float64

0    9.4
1    9.2
2    5.5
Name: nota_port, dtype: float64

0    6.0
1    6.0
2    6.0
Name: nota_port, dtype: float64

### Inglês

In [668]:
col = 'nota_ing'

extract_data(
	col,
	col_2022 = 'Inglês',
	col_2023 = 'Ing',
	col_2024 = 'Ing'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    6.0
1    9.7
2    6.9
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

---

## Mergendo dados

In [669]:
for year in dfs_yearly:
    dfs_yearly[year] = dfs_yearly[year].set_index('aluno')
dfs_yearly[2022] 

,ano_nascimento,inde,pedra_22,pedra_23,genero,ano_ingresso,instituicao_ensino,ian,ida,ieg,iaa,ips,ipp,ipv,nota_mat,nota_port,nota_ing
aluno,,,,,,,,,,,,,,,,,
Aluno-1,2003,5.783,Quartzo,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,2005,7.055,Ametista,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,2005,6.591,Ágata,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9
Aluno-4,2005,5.951,Quartzo,NaN,Masculino,2017,Rede decisão,10.0,5.0,4.5,8.8,5.6,NaN,5.278,2.8,3.5,8.7
Aluno-5,2005,7.427,Ametista,NaN,Feminino,2016,Rede decisão,10.0,5.2,8.6,7.9,5.6,NaN,7.389,7.0,2.9,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-856,2015,8.398,Topázio,NaN,Masculino,2022,Pública,10.0,8.4,9.1,10.0,7.5,NaN,8.000,8.5,8.2,NaN
Aluno-857,2013,8.154,Topázio,NaN,Feminino,2022,Pública,5.0,9.5,8.9,8.5,7.5,NaN,9.083,9.6,9.3,NaN
Aluno-858,2015,7.523,Ametista,NaN,Masculino,2022,Pública,10.0,7.7,7.1,10.0,5.6,NaN,7.333,8.4,6.9,NaN


### Atualizando alunos de 2022 até 2024

In [ ]:
data_2022 = raw_data[2022][fixed_columns].copy()
data_2022 = data_2022.set_index('nome_anonimizado')

for year in raw_data.keys():
    data_2022 = pd.merge(
        left = data_2022,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
data_2022.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ipp_2024,ipv_2024,pedra_2022_2024,pedra_2023_2024,pedra_2024_2024,pedra_2024_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1252,Feminino,2021,2002-06-03 03:00:00,9,10.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,INCLUIR,INCLUIR,NaN,NaN,NaN,Privada
Aluno-1660,Feminino,2021,2000-06-28 03:00:00,9,10.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,INCLUIR,INCLUIR,NaN,NaN,NaN,Bolsista Universitário *Formado (a)
Aluno-1661,Feminino,2021,2003-01-29 02:00:00,9,10.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,INCLUIR,INCLUIR,NaN,NaN,NaN,Bolsista Universitário *Formado (a)


In [ ]:
data_2022.shape

(1156, 51)

### Atualizando alunos de 2023 até 2024

In [ ]:
dados_2023 = raw_data[2023][fixed_columns].copy()
dados_2023 = dados_2023.set_index('nome_anonimizado')

old_students = data_2022.index.to_list()
current_students = dados_2023.index.to_list()

new_students = list(set(current_students) - set(old_students))

dados_2023 = dados_2023.loc[new_students]
dados_2023.head(3)

,genero,ano_ingresso,ano_nascimento
nome_anonimizado,,,


In [334]:
dados_2023['ano_nascimento'] = pd.to_datetime(dados_2023['ano_nascimento']).dt.year
dados_2023['ano_nascimento'] = dados_2023['ano_nascimento'].astype(int)

In [ ]:
for year in raw_data.keys():
    dados_2023 = pd.merge(
        left = dados_2023,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
dados_2023.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-948,Feminino,2023,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-1172,Feminino,2023,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.333333,6.063492,7.500,6.26,7.250000,6.196,0.0,5.0,5.0,Pública
Aluno-1201,Masculino,2023,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.000000,9.523810,8.751,7.51,8.333333,8.560,9.0,8.0,10.0,Pública


In [336]:
dados_2023.shape

(414, 39)

### Pegando os alunos de 2024

In [ ]:
dados_2024 = raw_data[2024][fixed_columns].copy()
dados_2024 = dados_2024.set_index('nome_anonimizado')

old_students = data_2022.index.to_list() + dados_2023.index.to_list()
current_students = dados_2024.index.to_list()

new_students = list(set(current_students) - set(old_students))

dados_2024 = dados_2024.loc[new_students]
dados_2024.head(3)

,genero,ano_ingresso,ano_nascimento
nome_anonimizado,,,
Aluno-1590,Feminino,2024,2009-02-28
Aluno-1377,Masculino,2024,2016-08-09
Aluno-1448,Feminino,2024,2011-10-28


In [338]:
dados_2024['ano_nascimento'] = pd.to_datetime(dados_2024['ano_nascimento']).dt.year
dados_2024['ano_nascimento'] = dados_2024['ano_nascimento'].astype(int)

In [ ]:
for year in raw_data.keys():
    dados_2024 = pd.merge(
        left = dados_2024,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
dados_2024.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1407,Masculino,2024,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.75,9.318182,10.002,7.51,7.812500,7.500,5.5,6.0,NaN,Pública
Aluno-1444,Masculino,2024,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.00,5.277778,8.502,7.51,7.708333,7.415,8.0,4.0,NaN,Pública
Aluno-1355,Masculino,2024,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.00,6.522727,10.002,7.51,8.125000,7.170,8.0,6.0,NaN,Pública


In [340]:
dados_2024.shape

(387, 39)

### Juntando as sheets

In [ ]:
df = pd.concat(
    [data_2022,
    dados_2023]
)

df = pd.concat(
    [df,
     dados_2024]
)

df.shape

(1661, 39)

## Tratando valores

In [342]:
df.dtypes

genero               object
ano_ingresso          int64
ano_nascimento        int64
fase_2022           float64
ian_2022            float64
ida_2022            float64
ieg_2022            float64
iaa_2022            float64
ips_2022            float64
ipp_2022            float64
ipv_2022            float64
matematica_2022     float64
portugues_2022      float64
ingles_2022         float64
instituicao_2022     object
fase_2023            object
ian_2023            float64
ida_2023            float64
ieg_2023            float64
iaa_2023            float64
ips_2023            float64
ipp_2023            float64
ipv_2023            float64
matematica_2023     float64
portugues_2023      float64
ingles_2023         float64
instituicao_2023     object
fase_2024            object
ian_2024            float64
ida_2024            float64
ieg_2024            float64
iaa_2024            float64
ips_2024            float64
ipp_2024            float64
ipv_2024            float64
matematica_2024     

### Strings

In [343]:
df.index

Index(['Aluno-1', 'Aluno-2', 'Aluno-3', 'Aluno-4', 'Aluno-5', 'Aluno-6',
       'Aluno-7', 'Aluno-8', 'Aluno-9', 'Aluno-10',
       ...
       'Aluno-1367', 'Aluno-1371', 'Aluno-1580', 'Aluno-1641', 'Aluno-1406',
       'Aluno-1290', 'Aluno-1420', 'Aluno-1407', 'Aluno-1444', 'Aluno-1355'],
      dtype='object', name='nome_anonimizado', length=1661)

In [344]:
df.index = df.index.str.capitalize()

In [345]:
cols_str = [
    'instituicao',
    'fase'
]

In [ ]:
for col in cols_str:
    for year in raw_data.keys():
        df[f'{col}_{year}'] = df[f'{col}_{year}'].astype(str)
        df[f'{col}_{year}'] = df[f'{col}_{year}'].str.lower()

### Floats

In [347]:
cols_float = [
    'ian',
    'ida',
    'ieg',
    'iaa',
    'ips',
    'ipp',
    'ipv',
    'matematica',
    'portugues',
    'ingles'
]

In [ ]:
for col in cols_float:
    for year in raw_data.keys():
        df[f'{col}_{year}'] = df[f'{col}_{year}'].astype(float)

### Ints

In [349]:
cols_int = [
    'ano_nascimento',
]

In [350]:
for col in cols_int:
    df[f'{col}'] = df[f'{col}'].astype(int)

In [351]:
df.dtypes

genero               object
ano_ingresso          int64
ano_nascimento        int64
fase_2022            object
ian_2022            float64
ida_2022            float64
ieg_2022            float64
iaa_2022            float64
ips_2022            float64
ipp_2022            float64
ipv_2022            float64
matematica_2022     float64
portugues_2022      float64
ingles_2022         float64
instituicao_2022     object
fase_2023            object
ian_2023            float64
ida_2023            float64
ieg_2023            float64
iaa_2023            float64
ips_2023            float64
ipp_2023            float64
ipv_2023            float64
matematica_2023     float64
portugues_2023      float64
ingles_2023         float64
instituicao_2023     object
fase_2024            object
ian_2024            float64
ida_2024            float64
ieg_2024            float64
iaa_2024            float64
ips_2024            float64
ipp_2024            float64
ipv_2024            float64
matematica_2024     

## Nulos

In [352]:
df = df.replace(
    'nan',
    np.nan
)

df = df.replace(
    'NA',
    np.nan
)

## Bases anuais

In [353]:
year = 2022
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2022 = df[cols]

year = 2023
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2023 = df[cols]

year = 2024
cols = fixed_columns[1:] + [ f'{col}_{year}' for col in yearly_columns ]
df_2024 = df[cols]

# Tratando dados
Adicionando novos parâmetros, tratando dados existentes

In [354]:
def drop_nulls(df):
    col = df.columns[-1:][0]
    to_drop = df[col].isna()
    to_drop = df[to_drop].index

    return df.drop(index = to_drop)

In [355]:
df_2022 = drop_nulls(df_2022)
df_2023 = drop_nulls(df_2023)
df_2024 = drop_nulls(df_2024)

In [356]:
_ = df_2024.instituicao_2024.isna()
_.value_counts()

instituicao_2024
False    1155
Name: count, dtype: int64

In [357]:
df.isna().sum()

genero                 0
ano_ingresso           0
ano_nascimento         0
fase_2022            801
ian_2022             801
ida_2022             801
ieg_2022             801
iaa_2022             801
ips_2022             801
ipp_2022             801
ipv_2022             801
matematica_2022      803
portugues_2022       803
ingles_2022         1378
instituicao_2022     801
fase_2023            647
ian_2023             647
ida_2023             724
ieg_2023             723
iaa_2023             710
ips_2023             716
ipp_2023             723
ipv_2023             723
matematica_2023      724
portugues_2023       724
ingles_2023         1327
instituicao_2023     647
fase_2024            505
ian_2024             505
ida_2024             606
ieg_2024             505
iaa_2024             607
ips_2024             607
ipp_2024             607
ipv_2024             607
matematica_2024      610
portugues_2024       611
ingles_2024         1187
instituicao_2024     506
dtype: int64

In [358]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ano_ingresso,1661.0,2021.821192,1.976655,2016.000000,2021.000000,2022.000000,2023.000000,2024.000000
ano_nascimento,1661.0,2010.526791,3.478857,1996.000000,2008.000000,2011.000000,2013.000000,2017.000000
ian_2022,860.0,6.424419,2.389609,2.500000,5.000000,5.000000,10.000000,10.000000
ida_2022,860.0,6.092907,2.046209,0.000000,4.800000,6.300000,7.600000,9.900000
ieg_2022,860.0,7.891163,1.638340,0.000000,7.000000,8.300000,9.100000,10.000000
iaa_2022,860.0,8.274419,2.064935,0.000000,7.900000,8.800000,9.500000,10.000000
ips_2022,860.0,6.905000,1.070707,2.500000,6.300000,7.500000,7.500000,10.000000
ipp_2022,860.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ipv_2022,860.0,7.253642,1.093383,2.500000,6.722000,7.333000,7.917000,10.000000
matematica_2022,858.0,5.806876,2.414215,0.000000,4.300000,6.000000,7.800000,10.000000


### Desistências

In [359]:
students_2022 = df_2022.index.to_list()
students_2023 = df_2023.index.to_list()
students_2024 = df_2024.index.to_list()

In [360]:
terminated_2022 = []
# student from 2022 is in 2023?
for student in students_2022:
    if student not in students_2023:
        terminated_2022.append(student)


terminated_2023 = []
# student from 2023 is in 2024?
for student in students_2023:
    if student not in students_2024:
        terminated_2023.append(student)

print(
    len(terminated_2022),
    len(terminated_2023),
    )

260 249


In [362]:
df['desistiu_no_ano_seguinte_2022'] = False
df_2022['desistiu_no_ano_seguinte_2022'] = False
df.loc[terminated_2022, 'desistiu_no_ano_seguinte_2022'] = True
df_2022.loc[terminated_2022, 'desistiu_no_ano_seguinte_2022'] = True


df['desistiu_no_ano_seguinte_2023'] = False
df_2023['desistiu_no_ano_seguinte_2023'] = False
df.loc[terminated_2023, 'desistiu_no_ano_seguinte_2023'] = True
df_2023.loc[terminated_2023, 'desistiu_no_ano_seguinte_2023'] = True

df_2023

,genero,ano_ingresso,ano_nascimento,fase_2023,ian_2023,ida_2023,ieg_2023,iaa_2023,ips_2023,ipp_2023,ipv_2023,matematica_2023,portugues_2023,ingles_2023,instituicao_2023,desistiu_no_ano_seguinte,desistiu_no_ano_seguinte_2023
nome_anonimizado,,,,,,,,,,,,,,,,,
Aluno-1,Menina,2016,2003,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-2,Menina,2017,2005,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-5,Menina,2016,2005,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,True
Aluno-6,Menina,2021,2004,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada - pagamento por *empresa parceira,False,False
Aluno-7,Menino,2017,2004,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-1022,Masculino,2023,2014,fase 1,10.0,4.7,8.5,8.5,3.77,7.18750,7.893333,4.7,4.7,NaN,pública,False,False
Aluno-1231,Masculino,2021,2001,fase 8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%,False,False
Aluno-948,Feminino,2023,2015,alfa,10.0,0.0,7.6,9.0,7.52,6.87500,8.340000,0.0,0.0,NaN,pública,False,True


## Médias por aluno

In [142]:
df.head(2)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1,Menina,2016,2003,7.0,5.0,4.0,4.1,8.3,5.6,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%
Aluno-2,Menina,2017,2005,7.0,10.0,6.8,5.2,8.8,6.3,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,privada *parcerias com bolsa 100%


In [144]:
df.ida_2022

nome_anonimizado
Aluno-1       4.0
Aluno-2       6.8
Aluno-3       5.6
Aluno-4       5.0
Aluno-5       5.2
             ... 
Aluno-1290    NaN
Aluno-1420    NaN
Aluno-1407    NaN
Aluno-1444    NaN
Aluno-1355    NaN
Name: ida_2022, Length: 1661, dtype: float64

## Export

In [ ]:
df.to_csv(FILE_JOINED_DATA)

df_2022.to_csv(FILE_JOINED_DATA2022)
df_2023.to_csv(FILE_JOINED_DATA2023)
df_2024.to_csv(FILE_JOINED_DATA2024)


# Conexão com o PostgreSQL

In [38]:
def connect_to_db(
        dbname = 'datathon',
        user = 'postgres',
        password_path = FILE_POSTGRES_PW,
        host = 'localhost',
        port = '5432'
):
    password = ''
    with open(password_path) as f:
        password = f.read()

    conn = ps.connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )

    return conn

In [39]:
cursor = None
conn = None
try:
    conn = connect_to_db()
    sql_create_schema = 'CREATE SCHEMA IF NOT EXISTS datathon;'
    
    cursor = conn.cursor()
    cursor.execute(sql_create_schema)
    conn.commit()

    print("Schema created successfully")
except Exception as e:
    
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



In [40]:
sql_types = {
    'object' : 'VARCHAR(255)',
    'int64': 'INT',
    'float64': 'FLOAT'
}

schema_types = {}

for col in df.columns:
    schema_types[col] = sql_types[f'{df[col].dtype}']

schema_types

{'genero': 'VARCHAR(255)',
 'ano_ingresso': 'INT',
 'ano_nascimento': 'INT',
 'fase_2022': 'VARCHAR(255)',
 'ian_2022': 'FLOAT',
 'ida_2022': 'FLOAT',
 'ieg_2022': 'FLOAT',
 'iaa_2022': 'FLOAT',
 'ips_2022': 'FLOAT',
 'ipp_2022': 'FLOAT',
 'ipv_2022': 'FLOAT',
 'matematica_2022': 'FLOAT',
 'portugues_2022': 'FLOAT',
 'ingles_2022': 'FLOAT',
 'instituicao_2022': 'VARCHAR(255)',
 'fase_2023': 'VARCHAR(255)',
 'ian_2023': 'FLOAT',
 'ida_2023': 'FLOAT',
 'ieg_2023': 'FLOAT',
 'iaa_2023': 'FLOAT',
 'ips_2023': 'FLOAT',
 'ipp_2023': 'FLOAT',
 'ipv_2023': 'FLOAT',
 'matematica_2023': 'FLOAT',
 'portugues_2023': 'FLOAT',
 'ingles_2023': 'FLOAT',
 'instituicao_2023': 'VARCHAR(255)',
 'fase_2024': 'VARCHAR(255)',
 'ian_2024': 'FLOAT',
 'ida_2024': 'FLOAT',
 'ieg_2024': 'FLOAT',
 'iaa_2024': 'FLOAT',
 'ips_2024': 'FLOAT',
 'ipp_2024': 'FLOAT',
 'ipv_2024': 'FLOAT',
 'matematica_2024': 'FLOAT',
 'portugues_2024': 'FLOAT',
 'ingles_2024': 'FLOAT',
 'instituicao_2024': 'VARCHAR(255)'}

In [41]:
sql_columns = ''
i = 0

for col in schema_types:
    col_type = schema_types[col]
    if i > 0:
        sql_columns += ', '
    sql_columns += f'{col} {col_type}'
    i += 1

sql_columns

'genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255)'

In [42]:
sql_create_table = f'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos({sql_columns})'
sql_create_table

'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos(genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255))'

In [43]:
try:
    conn = connect_to_db()
    cursor = conn.cursor()

    # 4. Execute the CREATE TABLE statement
    cursor.execute(sql_create_table)

    # 5. Commit the transaction
    conn.commit()
    print("Table created successfully!")
except Exception as e:
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    # 6. Close the cursor and connection
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



## Subindo dados

In [44]:
def ingest_data(file, table = 'datathon.passos_magicos'):
    conn = connect_to_db()
    cursor = conn.cursor()
    try:
        with open(file, 'r') as file:
            next(file) # skips header
            cursor.copy_expert(f'COPY {table} FROM STDIN WITH (FORMAT CSV)', file)
    except Exception as e:
        conn.rollback()
        print(f'Failed to insert | {e}')
    finally:
        conn.commit()
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

---

# Perguntas norteadoras

## 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

## 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

## 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

## 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

## 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

## 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

## 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

## 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

## 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

## 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

## 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

## Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?